# Asthma Prevalence / Vulnerability

## Cleaning and wrangling the data

The final product of this file is data.csv, which is the main data file used in the subsequent machine learning models.

In [1]:
# inspect dataset

In [2]:
!pip install geopandas
!pip install shapely

In [3]:
import pandas as pd
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import geopandas as gpd
from shapely.geometry import Point
from shapely.validation import make_valid
import numpy as np
import matplotlib.pyplot as plt

### Inspecting Allegheny County census tract file for future validation purposes.

Data taken from WPRDC should show all Census Tracts in year 2016. Result: there should be 402 rows per year in our final data set.

In [4]:
alCoCTs = pd.read_csv('our-data/AlleghenyCountyCensusTracts2016.csv')
print('Number of Census Tracts: ', len(alCoCTs))
alCoCTs.sort_values('geoid')

Number of Census Tracts:  402


,_id,fid,state_fp,county_fp,tract_ce,affgeoid,geoid,name,lsad,a_land,a_water,shape_length,shape_area
69,15019,70,42,3,10300,1400000US42003010300,42003010300,103.0,CT,866087,235982,0.055235,0.000117
368,15318,369,42,3,20100,1400000US42003020100,42003020100,201.0,CT,1678102,483177,0.083161,0.000231
28,14978,29,42,3,20300,1400000US42003020300,42003020300,203.0,CT,1594790,326537,0.077032,0.000204
70,15020,71,42,3,30500,1400000US42003030500,42003030500,305.0,CT,668309,0,0.046956,0.000072
394,15344,395,42,3,40200,1400000US42003040200,42003040200,402.0,CT,604717,0,0.049916,0.000064
...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,15162,213,42,3,981000,1400000US42003981000,42003981000,9810.0,CT,1372534,0,0.063284,0.000147
80,15030,81,42,3,981100,1400000US42003981100,42003981100,9811.0,CT,875989,0,0.043523,0.000093
68,15018,69,42,3,981200,1400000US42003981200,42003981200,9812.0,CT,460275,156913,0.037880,0.000067
165,15115,166,42,3,981800,1400000US42003981800,42003981800,9818.0,CT,1822889,310071,0.066922,0.000228


In [5]:
alCoCTs['geoid'].nunique()

402

In [6]:
alCoBGs = pd.read_csv('our-data/AlleghenyCountyCensusBlockGroups2016.csv')
print('Number of Block Groups: ', len(alCoBGs))
alCoBGs.sort_values('geoid')

Number of Block Groups:  1100


,_id,fid,state_fp,county_fp,tract_ce,blkgrp_ce,geoid,namelsad,mtfcc,funcstat,aland,awater,intptlat,intptlon,shape_length,shape_area
397,55277,398,42,3,10300,1,420030103001,Block Group 1,G5030,S,291362,235982,40.434050,-79.986430,0.052197,0.000056
55,54935,56,42,3,10300,2,420030103002,Block Group 2,G5030,S,275341,0,40.436894,-79.990472,0.022431,0.000029
775,55655,776,42,3,10300,3,420030103003,Block Group 3,G5030,S,168360,0,40.436893,-79.983761,0.020349,0.000018
178,55058,179,42,3,10300,4,420030103004,Block Group 4,G5030,S,131024,0,40.437347,-79.977287,0.020653,0.000014
398,55278,399,42,3,20100,1,420030201001,Block Group 1,G5030,S,435122,249581,40.439695,-80.006178,0.046121,0.000073
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,55048,169,42,3,981000,1,420039810001,Block Group 1,G5030,S,1372534,0,40.473101,-79.948865,0.066651,0.000146
206,55086,207,42,3,981100,1,420039811001,Block Group 1,G5030,S,875989,0,40.442038,-79.906326,0.046587,0.000093
748,55628,749,42,3,981200,1,420039812001,Block Group 1,G5030,S,460275,156913,40.446463,-80.011958,0.038948,0.000066
139,55019,140,42,3,981800,1,420039818001,Block Group 1,G5030,S,1822889,310071,40.477494,-79.896498,0.068401,0.000227


In [7]:
alCoBGs['geoid'].nunique()

1100

### Prevalence Data

In [8]:
#Loading in 2020 data to fill in missing rows and filtering on Allegheny County values
placesData = pd.read_csv('our-data/PLACES__Census_Tract_Data__GIS_Friendly_Format___2020_release_20250417.csv')
placesData = placesData[placesData['CountyName']=='Allegheny']

#Loading in data for 2019
cities2019 = pd.read_csv("our-data/500_Cities__Census_Tract-level_Data__GIS_Friendly_Format___2019_release_20250405.csv")
#Filtering for Pittsburgh census tracts
cities2019=cities2019[cities2019['PlaceName'] == 'Pittsburgh']
#Removing values from the 2020 data set that are already in the 2019 dataset
places2019 = placesData[placesData['TractFIPS'].apply(lambda x: x not in cities2019['TractFIPS'].to_list())]
#Joining to represent all Census Tracts in Allegheny County and adding column 'Year' indicating time period that each row represents
prevn2019 = pd.concat([cities2019, places2019])
prevn2019['Year'] = ['2019']*len(prevn2019)

#Above process is repeated for the remaining years
cities2018 = pd.read_csv("our-data/500_Cities__Census_Tract-level_Data__GIS_Friendly_Format___2018_release_20250416.csv")
cities2018=cities2018[cities2018['PlaceName'] == 'Pittsburgh']
places2018= placesData[placesData['TractFIPS'].apply(lambda x: x not in cities2018['TractFIPS'].to_list())]
prevn2018 = pd.concat([cities2018, places2018])
prevn2018['Year'] = ['2018']*len(prevn2018)

cities2017 = pd.read_csv("our-data/500_Cities__Census_Tract-level_Data__GIS_Friendly_Format___2017_release_20250416.csv")
cities2017=cities2017[cities2017['PlaceName'] == 'Pittsburgh']
places2017= placesData[placesData['TractFIPS'].apply(lambda x: x not in cities2017['TractFIPS'].to_list())]
prevn2017 = pd.concat([cities2017, places2017])
prevn2017['Year'] = ['2017']*len(prevn2017)

cities2016 = pd.read_csv("our-data/500_Cities__Census_Tract-level_Data__GIS_Friendly_Format___2016_release_20250416.csv")
cities2016=cities2016[cities2016['PlaceName'] == 'Pittsburgh']
places2016= placesData[placesData['TractFIPS'].apply(lambda x: x not in cities2016['TractFIPS'].to_list())]
prevn2016 = pd.concat([cities2016, places2016])
prevn2016['Year'] = ['2016']*len(prevn2016)

#Data for each year is joined together after every year is completed
cond_prevn = pd.concat([prevn2019, prevn2018, prevn2017, prevn2016])
cond_prevn

,StateAbbr,PlaceName,PlaceFIPS,TractFIPS,Place_TractID,Population2010,ACCESS2_CrudePrev,ACCESS2_Crude95CI,ARTHRITIS_CrudePrev,ARTHRITIS_Crude95CI,BINGE_CrudePrev,BINGE_Crude95CI,BPHIGH_CrudePrev,BPHIGH_Crude95CI,BPMED_CrudePrev,BPMED_Crude95CI,CANCER_CrudePrev,CANCER_Crude95CI,CASTHMA_CrudePrev,CASTHMA_Crude95CI,CHD_CrudePrev,CHD_Crude95CI,CHECKUP_CrudePrev,CHECKUP_Crude95CI,CHOLSCREEN_CrudePrev,CHOLSCREEN_Crude95CI,COLON_SCREEN_CrudePrev,COLON_SCREEN_Crude95CI,COPD_CrudePrev,COPD_Crude95CI,COREM_CrudePrev,COREM_Crude95CI,COREW_CrudePrev,COREW_Crude95CI,CSMOKING_CrudePrev,CSMOKING_Crude95CI,DENTAL_CrudePrev,DENTAL_Crude95CI,DIABETES_CrudePrev,DIABETES_Crude95CI,HIGHCHOL_CrudePrev,HIGHCHOL_Crude95CI,KIDNEY_CrudePrev,KIDNEY_Crude95CI,LPA_CrudePrev,LPA_Crude95CI,MAMMOUSE_CrudePrev,MAMMOUSE_Crude95CI,MHLTH_CrudePrev,MHLTH_Crude95CI,OBESITY_CrudePrev,OBESITY_Crude95CI,PAPTEST_CrudePrev,PAPTEST_Crude95CI,PHLTH_CrudePrev,PHLTH_Crude95CI,SLEEP_CrudePrev,SLEEP_Crude95CI,STROKE_CrudePrev,STROKE_Crude95CI,TEETHLOST_CrudePrev,TEETHLOST_Crude95CI,Geolocation,StateDesc,CountyName,CountyFIPS,TotalPopulation,CERVICAL_CrudePrev,CERVICAL_Crude95CI,Year,population_count
450,PA,Pittsburgh,4261000.0,42003010300,4261000-42003010300,6600.0,12.9,"(10.2, 16.9)",11.5,"(10.9, 12.3)",27.4,"(25.7, 29.2)",19.0,"(18.0, 19.9)",51.9,"(50.2, 53.6)",1.8,"( 1.7, 1.9)",11.1,"(10.1, 12.9)",2.6,"( 2.4, 2.8)",63.7,"(61.7, 65.6)",58.1,"(53.6, 62.0)",50.5,"(46.7, 53.8)",4.4,"( 3.8, 5.4)",38.8,"(33.8, 44.1)",30.5,"(26.4, 34.5)",26.5,"(21.9, 31.4)",49.6,"(42.3, 55.9)",5.0,"( 4.7, 5.3)",17.6,"(16.8, 18.6)",1.8,"( 1.7, 1.9)",24.9,"(21.5, 28.6)",80.0,"(77.6, 82.0)",20.1,"(17.0, 23.5)",29.8,"(27.9, 31.3)",75.0,"(69.2, 80.4)",10.0,"( 8.8, 11.8)",42.3,"(39.5, 44.4)",1.6,"( 1.5, 1.8)",24.3,"(19.1, 30.4)","(40.43572601140, -79.9851241394)",NaN,NaN,NaN,NaN,NaN,NaN,2019,NaN
451,PA,Pittsburgh,4261000.0,42003020100,4261000-42003020100,3629.0,6.7,"( 5.5, 8.5)",17.7,"(17.0, 18.4)",25.7,"(24.9, 26.6)",22.1,"(21.4, 22.8)",71.8,"(70.5, 73.0)",4.5,"( 4.4, 4.7)",9.3,"( 8.8, 9.9)",3.9,"( 3.6, 4.2)",68.4,"(67.4, 69.3)",73.4,"(70.9, 75.4)",65.8,"(63.5, 67.9)",3.8,"( 3.4, 4.4)",50.2,"(45.9, 54.4)",40.9,"(37.7, 44.0)",14.4,"(11.9, 17.1)",68.1,"(63.7, 71.6)",6.4,"( 6.1, 6.8)",24.9,"(24.3, 25.7)",2.0,"( 1.9, 2.1)",17.3,"(15.5, 19.3)",80.1,"(77.9, 81.8)",12.2,"(10.7, 14.1)",23.4,"(22.3, 24.7)",80.2,"(77.2, 82.8)",7.9,"( 7.2, 8.7)",35.9,"(34.0, 37.8)",1.9,"( 1.8, 2.1)",11.0,"( 8.4, 13.8)","(40.44068353060, -80.0000351595)",NaN,NaN,NaN,NaN,NaN,NaN,2019,NaN
452,PA,Pittsburgh,4261000.0,42003020300,4261000-42003020300,616.0,4.8,"( 4.1, 5.8)",16.8,"(16.2, 17.5)",27.2,"(26.5, 28.0)",19.7,"(19.1, 20.4)",67.6,"(66.4, 68.5)",4.5,"( 4.3, 4.6)",8.2,"( 7.9, 8.5)",2.8,"( 2.6, 3.0)",67.4,"(66.6, 68.2)",80.4,"(78.9, 81.4)",70.0,"(67.9, 71.6)",2.8,"( 2.5, 3.2)",54.3,"(49.8, 58.7)",46.1,"(43.0, 48.8)",12.3,"(10.5, 15.0)",75.9,"(73.1, 77.8)",4.9,"( 4.7, 5.2)",24.2,"(23.7, 24.8)",1.6,"( 1.6, 1.7)",14.1,"(13.0, 15.7)",83.5,"(81.5, 85.0)",9.3,"( 8.6, 10.4)",23.4,"(22.5, 24.6)",86.3,"(84.6, 87.5)",6.5,"( 5.9, 7.2)",34.6,"(33.4, 36.1)",1.4,"( 1.3, 1.5)",5.3,"( 4.0, 7.5)","(40.45425423370, -79.9814998549)",NaN,NaN,NaN,NaN,NaN,NaN,2019,NaN
453,PA,Pittsburgh,4261000.0,42003030500,4261000-42003030500,2256.0,14.4,"(11.9, 17.2)",35.6,"(34.1, 37.0)",13.5,"(13.0, 14.0)",48.3,"(47.0, 49.5)",84.3,"(83.6, 85.0)",6.9,"( 6.4, 7.1)",12.8,"(11.9, 13.7)",9.9,"( 8.9, 11.0)",79.2,"(78.6, 79.9)",79.7,"(78.1, 81.3)",56.7,"(52.5, 60.5)",10.6,"( 9.0, 12.2)",32.5,"(27.0, 38.3)",28.4,"(23.4, 33.7)",26.9,"(23.6, 30.2)",42.8,"(38.6, 47.0)",20.5,"(19.0, 21.9)",37.2,"(36.1, 38.3)",5.3,"( 4.9, 5.7)",36.9,"(33.8, 39.8)",81.4,"(78.2, 84.2)",17.2,"(15.5, 18.9)",42.6,"(41.1, 44.0)",82.7,"(80.2, 85.1)",18.8,"(16.8, 20.8)",47.5,"(45.9, 49.0)",7.1,"( 6.2, 8.1)",30.2,"(21.5, 39.5)","(40.44273346300, -79.9837719110)",NaN,NaN,NaN,NaN,NaN,NaN,2019,NaN
454,PA,Pittsburgh,4261000.0,42003040200,4261000-42003040200,2604.0,11.8,"( 8.8, 15.8)",16.5,"(15.7, 17.4)",21

In [9]:
print("Unique tract IDs: " + str(cond_prevn['TractFIPS'].nunique()))
print("Shape: " + str(cond_prevn.shape))

Unique tract IDs: 392
Shape: (1568, 71)


We are missing data for 10 tracts for all years (final row count is 392 * 4 = 1568) there 10 tracts are uninhabited (i.e. Parks, Industrial Zones, etc.)

In [10]:
cond_prevn = cond_prevn.rename(columns={'TractFIPS': 'Geo_FIPS'})
#Removing unrelated prevalence columns (not comorbidities with Asthma)
def drop_columns_with_keyword(df, keyword):
    # Identify columns containing the keyword (95CI)
    cols_to_drop = [col for col in df.columns if keyword.lower() in col.lower()]
    # Drop them and return the updated DataFrame
    return df.drop(columns=cols_to_drop)
cond_prevn = drop_columns_with_keyword(cond_prevn,'95CI')
cond_prevn = cond_prevn.drop([
    'ARTHRITIS_CrudePrev',
    'BINGE_CrudePrev',
    'CANCER_CrudePrev',
    'COLON_SCREEN_CrudePrev',
    'DENTAL_CrudePrev',
    'KIDNEY_CrudePrev',
    'LPA_CrudePrev',
    'MAMMOUSE_CrudePrev',
    'MHLTH_CrudePrev',
    'PAPTEST_CrudePrev',
    'PHLTH_CrudePrev',
    'SLEEP_CrudePrev',
    'STROKE_CrudePrev',
    'TEETHLOST_CrudePrev'], axis=1)
cond_prevn['lat'] = cond_prevn['Geolocation'].str[1:15]
cond_prevn['lon'] = cond_prevn['Geolocation'].str[-15:-1]
cond_prevn

,StateAbbr,PlaceName,PlaceFIPS,Geo_FIPS,Place_TractID,Population2010,ACCESS2_CrudePrev,BPHIGH_CrudePrev,BPMED_CrudePrev,CASTHMA_CrudePrev,CHD_CrudePrev,CHECKUP_CrudePrev,CHOLSCREEN_CrudePrev,COPD_CrudePrev,COREM_CrudePrev,COREW_CrudePrev,CSMOKING_CrudePrev,DIABETES_CrudePrev,HIGHCHOL_CrudePrev,OBESITY_CrudePrev,Geolocation,StateDesc,CountyName,CountyFIPS,TotalPopulation,CERVICAL_CrudePrev,Year,population_count,lat,lon
450,PA,Pittsburgh,4261000.0,42003010300,4261000-42003010300,6600.0,12.9,19.0,51.9,11.1,2.6,63.7,58.1,4.4,38.8,30.5,26.5,5.0,17.6,29.8,"(40.43572601140, -79.9851241394)",NaN,NaN,NaN,NaN,NaN,2019,NaN,40.43572601140,-79.9851241394
451,PA,Pittsburgh,4261000.0,42003020100,4261000-42003020100,3629.0,6.7,22.1,71.8,9.3,3.9,68.4,73.4,3.8,50.2,40.9,14.4,6.4,24.9,23.4,"(40.44068353060, -80.0000351595)",NaN,NaN,NaN,NaN,NaN,2019,NaN,40.44068353060,-80.0000351595
452,PA,Pittsburgh,4261000.0,42003020300,4261000-42003020300,616.0,4.8,19.7,67.6,8.2,2.8,67.4,80.4,2.8,54.3,46.1,12.3,4.9,24.2,23.4,"(40.45425423370, -79.9814998549)",NaN,NaN,NaN,NaN,NaN,2019,NaN,40.45425423370,-79.9814998549
453,PA,Pittsburgh,4261000.0,42003030500,4261000-42003030500,2256.0,14.4,48.3,84.3,12.8,9.9,79.2,79.7,10.6,32.5,28.4,26.9,20.5,37.2,42.6,"(40.44273346300, -79.9837719110)",NaN,NaN,NaN,NaN,NaN,2019,NaN,40.44273346300,-79.9837719110
454,PA,Pittsburgh,4261000.0,42003040200,4261000-42003040200,2604.0,11.8,23.3,67.1,11.7,4.0,67.5,61.4,5.5,31.8,28.0,22.3,8.0,22.3,29.3,"(40.44011022030, -79.9663476233)",NaN,NaN,NaN,NaN,NaN,2019,NaN,40.44011022030,-79.9663476233
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55202,PA,NaN,NaN,42003564000,NaN,NaN,8.3,28.1,74.5,9.9,5.6,77.1,85.8,6.4,43.9,33.8,19.7,8.4,32.3,31.9,POINT (-80.2402315 40.41285776),Pennsylvania,Allegheny,42003.0,5469.0,86.9,2016,NaN,OINT (-80.2402,15 40.41285776
55203,PA,NaN,NaN,42003564100,NaN,NaN,4.3,28.8,79.0,8.7,5.4,81.1,91.1,4.7,50.8,41.1,10.8,8.0,34.8,28.1,POINT (-80.08698899 40.42759244),Pennsylvania,Allegheny,42003.0,882.0,89.8,2016,NaN,OINT (-80.0869,99 40.42759244
55204,PA,NaN,NaN,42003564200,NaN,NaN,7.6,33.4,79.7,9.6,7.2,80.0,88.0,7.4,43.7,34.6,18.1,10.3,36.6,31.5,POINT (-79.79426581 40.33230116),Pennsylvania,Allegheny,42003.0,2224.0,86.1,2016,NaN,OINT (-79.7942,81 40.33230116
55205,PA,NaN,NaN,42003564400,NaN,NaN,8.7,32.8,78.5,10.1,7.2,79.3,86.8,7.7,42.5,32.5,20.1,10.4,35.5,32.7,POINT (-79.79127061 40.37623919),Pennsylvania,Allegheny,42003.0,5620.0,85.2,2016,NaN,OINT (-79.7912,61 40.37623919


In [11]:
print(cond_prevn.shape)
cond_prevn.isna().sum()

(1568, 30)


StateAbbr                  0
PlaceName               1060
PlaceFIPS               1060
Geo_FIPS                   0
Place_TractID           1060
Population2010          1187
ACCESS2_CrudePrev          0
BPHIGH_CrudePrev           0
BPMED_CrudePrev            0
CASTHMA_CrudePrev          0
CHD_CrudePrev              0
CHECKUP_CrudePrev          0
CHOLSCREEN_CrudePrev       0
COPD_CrudePrev             0
COREM_CrudePrev            0
COREW_CrudePrev            8
CSMOKING_CrudePrev         0
DIABETES_CrudePrev         0
HIGHCHOL_CrudePrev         0
OBESITY_CrudePrev          0
Geolocation                0
StateDesc                508
CountyName               508
CountyFIPS               508
TotalPopulation          508
CERVICAL_CrudePrev       508
Year                       0
population_count        1441
lat                        0
lon                        0
dtype: int64

After dropping non-relevant conditions, we have 1568 rows and 30 columns. However, there are still columns with a significant amount of missing values. This is due to the 2020 and 2016-19 datasets having slight differences in the columns they contained. Thus, we will drop these columns as they are not relevant to our analysis.

In [12]:
#Renaming dataframe to be more descriptive
prevalence_data = cond_prevn

#Dropping columns containing NaNs
prevalence_data = prevalence_data.drop(columns=["COREW_CrudePrev", 'StateAbbr', 'PlaceName',
                                                'PlaceFIPS', 'Place_TractID', 'StateDesc', 'CountyName',
                                                'CountyFIPS', 'TotalPopulation', 'CERVICAL_CrudePrev',
                                                'Population2010', 'population_count'])

# Keeping only census tract level identifier for Geo_FIPS to facilitate later joins
prevalence_data['Geo_FIPS'] = prevalence_data['Geo_FIPS'].apply(lambda x: int(str(x)[-6:]))

#Dataframe is now free of missing values
prevalence_data.isna().sum()

Geo_FIPS                0
ACCESS2_CrudePrev       0
BPHIGH_CrudePrev        0
BPMED_CrudePrev         0
CASTHMA_CrudePrev       0
CHD_CrudePrev           0
CHECKUP_CrudePrev       0
CHOLSCREEN_CrudePrev    0
COPD_CrudePrev          0
COREM_CrudePrev         0
CSMOKING_CrudePrev      0
DIABETES_CrudePrev      0
HIGHCHOL_CrudePrev      0
OBESITY_CrudePrev       0
Geolocation             0
Year                    0
lat                     0
lon                     0
dtype: int64

### Utilization Data

In [13]:
ER_use = pd.read_csv("our-data/Allegheny_county_ER_use.csv")
ER_use

,_id,Geo_FIPS,StateFIPS,CountyFIPS,TractFIPS,BlockgroupFIPS,TotalPopEst2015_19ACS,YearOfStartDate,UnderAge1PopEst2015_19ACS,Age0to17PopEst2015_19ACS,Unique0to17WithED_Visit,Unique0to17WithED_VisitPer100,NumberED_VisitsAge0to17,ED_VisitsPer100Age0to17,NumberLowAcuityED_VisitsAge0to17,LowAcuityED_VisitsPer100Age0to17,NumberAsthmaRelatedED_Visits,AsthmaRelatedED_VisitsPer100Age0to17,NumberED_VisitsByChildrenUnder1YearOld,NumberOfInjuryRelatedVisits,InjuryRelatedEDVisitsPer100Age0to17,NumberAcuteRespiratoryTractInfectionRelatedED_VisitsAge0to17,AcuteRespiratoryTractInfectionRelatedED_VisitsPer100Age0to17
0,1,420031301002,42,3,130100,2,518,2016.0,9.4,207,70.0,33.8,122.0,58.9,62.0,30.0,40.0,19.3,14.0,1.0,0.5,18.0,8.7
1,2,420031301002,42,3,130100,2,518,2017.0,9.4,207,69.0,33.3,130.0,62.8,80.0,38.6,54.0,26.1,11.0,1.0,0.5,27.0,13.0
2,3,420031301002,42,3,130100,2,518,2018.0,9.4,207,66.0,31.9,138.0,66.7,85.0,41.1,59.0,28.5,13.0,2.0,1.0,19.0,9.2
3,4,420031301002,42,3,130100,2,518,2019.0,9.4,207,66.0,31.9,148.0,71.5,96.0,46.4,68.0,32.9,7.0,NaN,0.0,27.0,13.0
4,5,420031301003,42,3,130100,3,489,2016.0,0.0,134,38.0,28.4,71.0,53.0,39.0,29.1,13.0,9.7,12.0,2.0,1.5,10.0,7.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4365,4366,420039818001,42,3,981800,1,207,2019.0,0.0,47,4.0,8.5,8.0,17.0,2.0,4.3,NaN,0.0,NaN,NaN,0.0,NaN,0.0
4366,4367,420039822001,42,3,982200,1,4618,2016.0,0.0,80,6.0,7.5,10.0,12.5,2.0,2.5,NaN,0.0,1.0,NaN,0.0,NaN,0.0
4367,4368,420039822001,42,3,982200,1,4618,2017.0,0.0,80,10.0,12.5,10.0,12.5,2.0,2.5,1.0,1.3,1.0,NaN,0.0,3.0,3.8
4368,4369,420039822001,42,3,982200,1,4618,2018.0,0.0,80,7.0,8.8,9.0,11.3,2.0,2.5,NaN,0.0,2.0,NaN,0.0,3.0,3.8


In [14]:
ER_use.shape

(4370, 23)

In [15]:
PC_use = pd.read_csv("our-data/Allegheny_county_PC_use.csv")
# PC_use = PC_use.fillna(0)
PC_use

,_id,Geo_FIPS,StateFIPS,CountyFIPS,TractFIPS,BlockgroupFIPS,TotalPopEst2015_19ACS,Age0to17PopEst2015_19ACS,YearOfContactDate,AllPrimaryCarePatientsAge0to17,WellChildVisitsInPastYearAge0to17,WellChildVisitsInPastYearPer100PrimaryCarePatients,AsthmaDiagnosisAge0to17,AsthmaDiagnosisPer100PrimaryCarePatientsAge0to17
0,1,420031301002,42,3,130100,2,518,207,2016,82,59,72.0,17.0,20.7
1,2,420031301002,42,3,130100,2,518,207,2017,94,58,61.7,19.0,20.2
2,3,420031301002,42,3,130100,2,518,207,2018,89,63,70.8,19.0,21.3
3,4,420031301002,42,3,130100,2,518,207,2019,86,61,70.9,17.0,19.8
4,5,420031301003,42,3,130100,3,489,134,2016,53,34,64.2,18.0,34.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4361,4362,420039818001,42,3,981800,1,207,47,2019,4,3,75.0,2.0,50.0
4362,4363,420039822001,42,3,982200,1,4618,80,2016,16,13,81.3,1.0,6.3
4363,4364,420039822001,42,3,982200,1,4618,80,2017,18,16,88.9,1.0,5.6
4364,4365,420039822001,42,3,982200,1,4618,80,2018,19,14,73.7,1.0,5.3


In [16]:
PC_use.shape

(4366, 14)

`ER_use` has 4370 rows and 23 columns

`PC_use` has 4366 rows and 14 columns

Neither of these are divisible by 4, so we are missing some years for some tracts/blockgroups

In [17]:
ER_use['TractFIPS'].nunique() # 402 tracts in ER data
PC_use['TractFIPS'].nunique() # 397 tracts in PC data

# Get number of occurrences of each tract + blockgroup combination; we want to see each combination appear 4 times (for the four years 2016-19)
ER_combo_counts = ER_use.groupby(['TractFIPS', 'BlockgroupFIPS']).size().reset_index(name = 'Count').sort_values('Count', ascending = False)
ER_combo_counts.tail(20)

PC_combo_counts = PC_use.groupby(['TractFIPS', 'BlockgroupFIPS']).size().reset_index(name = 'Count').sort_values('Count', ascending = False)

In both cases, most tract + blockgroup combinations have all four years of data. For ER use:

* BGs 40400-1, 562000-2, 10300-2, and 70500-2 have 3 years of data;
* 563200-1 has 2 years of data;
* 980800-1, 980700-1, 980600-1, 980000-1, 980900-1, 981100-1, 981200-1, and 10300-1 all have 1 year of data

For PC use:

* BG 10300-2 has 3 years of data;
* 563800-4 has 2 years of data;
* 980000-1 has 1 year of data

In [18]:
# Bring in all Allegheny County blockgroups so we can ensure we have all of them
alCoBGs.head()
alCoBGs_subset = alCoBGs[['geoid', 'tract_ce', 'blkgrp_ce']]
alCoBGs_subset.head()
alCoBGs_subset['tract_ce'].nunique()

402

In [19]:
ER_use[ER_use['TractFIPS'] == 981100]
ER_use[ER_use['YearOfStartDate'].isna()]

PC_use[PC_use['YearOfContactDate'].isna()]

,_id,Geo_FIPS,StateFIPS,CountyFIPS,TractFIPS,BlockgroupFIPS,TotalPopEst2015_19ACS,Age0to17PopEst2015_19ACS,YearOfContactDate,AllPrimaryCarePatientsAge0to17,WellChildVisitsInPastYearAge0to17,WellChildVisitsInPastYearPer100PrimaryCarePatients,AsthmaDiagnosisAge0to17,AsthmaDiagnosisPer100PrimaryCarePatientsAge0to17


In [20]:
# Add placeholder rows for tracts + blockgroups that are missing
# First need to add years 2016-19 to base blockgroup data frame
years = pd.DataFrame({'year': [2016, 2017, 2018, 2019]})
alCoBGs_subset['key'] = 1 # Use key columns to join
years['key'] = 1
base_df = alCoBGs_subset.merge(years, on = 'key').drop(columns = 'key')

# Merge base data frame containing all tracts, blockgroups, and years with the ER use dataset
ER_expanded = base_df.merge(ER_use, how = 'left', left_on = ['tract_ce', 'blkgrp_ce', 'year'], right_on = ['TractFIPS', 'BlockgroupFIPS', 'YearOfStartDate']).sort_values(by = '_id')
ER_expanded.tail(50)
ER_expanded['Geo_FIPS'] = ER_expanded['Geo_FIPS'].fillna(ER_expanded['geoid'])
ER_expanded['TractFIPS'] = ER_expanded['TractFIPS'].fillna(ER_expanded['tract_ce'])
ER_expanded['BlockgroupFIPS'] = ER_expanded['BlockgroupFIPS'].fillna(ER_expanded['blkgrp_ce'])
ER_expanded['YearOfStartDate'] = ER_expanded['YearOfStartDate'].fillna(ER_expanded['year'])
ER_expanded['StateFIPS'] = ER_expanded['StateFIPS'].fillna(42)
ER_expanded['CountyFIPS'] = ER_expanded['CountyFIPS'].fillna(3)
ER_expanded.shape

# Merge base + ER data frame with the PC use dataset
merged_data = ER_expanded.merge(PC_use, how = 'left', left_on = ['tract_ce', 'blkgrp_ce', 'year'], right_on = ['TractFIPS', 'BlockgroupFIPS', 'YearOfContactDate'], suffixes=('_er', '_pc'))
merged_data.head(13)
util_data = merged_data.drop(columns = ['tract_ce', 'blkgrp_ce', 'year', '_id_er', '_id_pc', 'Geo_FIPS_pc', 'StateFIPS_pc', 'CountyFIPS_pc', 'TractFIPS_pc', 'BlockgroupFIPS_pc', 'TotalPopEst2015_19ACS_pc', 'Age0to17PopEst2015_19ACS_pc'])
util_data = util_data.rename(columns = {'Geo_FIPS_er':'Geo_FIPS', 'StateFIPS_er':'StateFIPS', 'CountyFIPS_er':'CountyFIPS', 'TractFIPS_er':'TractFIPS', 'BlockgroupFIPS_er':'BlockgroupFIPS', 'TotalPopEst2015_19ACS_er':'TotalPopEst2015_19ACS'})
util_data.shape

C:\Users\Mahima Batheja\AppData\Local\Temp\ipykernel_37016\1826672544.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alCoBGs_subset['key'] = 1 # Use key columns to join


(4400, 29)

In [21]:
#merging utilization data for Asthma ER and primary care
util_data = PC_use.merge(ER_use, how = 'left', left_on = ['Geo_FIPS', 'YearOfContactDate'], right_on = ['Geo_FIPS', 'YearOfStartDate'], validate='one_to_one')
#dropping repeat columns
util_data = util_data.drop(["_id_y","StateFIPS_y","CountyFIPS_y","TractFIPS_y","BlockgroupFIPS_y",
                            "TotalPopEst2015_19ACS_y","Age0to17PopEst2015_19ACS_y"],axis="columns")
util_data

,_id_x,Geo_FIPS,StateFIPS_x,CountyFIPS_x,TractFIPS_x,BlockgroupFIPS_x,TotalPopEst2015_19ACS_x,Age0to17PopEst2015_19ACS_x,YearOfContactDate,AllPrimaryCarePatientsAge0to17,WellChildVisitsInPastYearAge0to17,WellChildVisitsInPastYearPer100PrimaryCarePatients,AsthmaDiagnosisAge0to17,AsthmaDiagnosisPer100PrimaryCarePatientsAge0to17,YearOfStartDate,UnderAge1PopEst2015_19ACS,Unique0to17WithED_Visit,Unique0to17WithED_VisitPer100,NumberED_VisitsAge0to17,ED_VisitsPer100Age0to17,NumberLowAcuityED_VisitsAge0to17,LowAcuityED_VisitsPer100Age0to17,NumberAsthmaRelatedED_Visits,AsthmaRelatedED_VisitsPer100Age0to17,NumberED_VisitsByChildrenUnder1YearOld,NumberOfInjuryRelatedVisits,InjuryRelatedEDVisitsPer100Age0to17,NumberAcuteRespiratoryTractInfectionRelatedED_VisitsAge0to17,AcuteRespiratoryTractInfectionRelatedED_VisitsPer100Age0to17
0,1,420031301002,42,3,130100,2,518,207,2016,82,59,72.0,17.0,20.7,2016.0,9.4,70.0,33.8,122.0,58.9,62.0,30.0,40.0,19.3,14.0,1.0,0.5,18.0,8.7
1,2,420031301002,42,3,130100,2,518,207,2017,94,58,61.7,19.0,20.2,2017.0,9.4,69.0,33.3,130.0,62.8,80.0,38.6,54.0,26.1,11.0,1.0,0.5,27.0,13.0
2,3,420031301002,42,3,130100,2,518,207,2018,89,63,70.8,19.0,21.3,2018.0,9.4,66.0,31.9,138.0,66.7,85.0,41.1,59.0,28.5,13.0,2.0,1.0,19.0,9.2
3,4,420031301002,42,3,130100,2,518,207,2019,86,61,70.9,17.0,19.8,2019.0,9.4,66.0,31.9,148.0,71.5,96.0,46.4,68.0,32.9,7.0,NaN,0.0,27.0,13.0
4,5,420031301003,42,3,130100,3,489,134,2016,53,34,64.2,18.0,34.0,2016.0,0.0,38.0,28.4,71.0,53.0,39.0,29.1,13.0,9.7,12.0,2.0,1.5,10.0,7.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4361,4362,420039818001,42,3,981800,1,207,47,2019,4,3,75.0,2.0,50.0,2019.0,0.0,4.0,8.5,8.0,17.0,2.0,4.3,NaN,0.0,NaN,NaN,0.0,NaN,0.0
4362,4363,420039822001,42,3,982200,1,4618,80,2016,16,13,81.3,1.0,6.3,2016.0,0.0,6.0,7.5,10.0,12.5,2.0,2.5,NaN,0.0,1.0,NaN,0.0,NaN,0.0
4363,4364,420039822001,42,3,982200,1,4618,80,2017,18,16,88.9,1.0,5.6,2017.0,0.0,10.0,12.5,10.0,12.5,2.0,2.5,1.0,1.3,1.0,NaN,0.0,3.0,3.8
4364,4365,420039822001,42,3,982200,1,4618,80,2018,19,14,73.7,1.0,5.3,2018.0,0.0,7.0,8.8,9.0,11.3,2.0,2.5,NaN,0.0,2.0,NaN,0.0,3.0,3.8


In [22]:
util_data.shape

(4366, 29)

In [23]:
#Joining rows based on CTs
#Dropping Per100 columns
util_data = drop_columns_with_keyword(util_data,'Per100')
util_data['Geo_FIPS'] = util_data['Geo_FIPS'].apply(str)
util_data['Geo_FIPS'] = util_data['Geo_FIPS'].str[:-1]
#Changing Geo_FIPS to contain only Census Tract identifier
util_data['Geo_FIPS'] = util_data['Geo_FIPS'].apply(lambda x: int(x[-6:]))
util_data = util_data.drop(columns=['_id_x', 'StateFIPS_x', 'CountyFIPS_x', 'TractFIPS_x', 'BlockgroupFIPS_x'])
util_data = util_data.groupby(['Geo_FIPS', 'YearOfContactDate'], as_index=False).sum()
util_data.isna().sum()

Geo_FIPS                                                        0
YearOfContactDate                                               0
TotalPopEst2015_19ACS_x                                         0
Age0to17PopEst2015_19ACS_x                                      0
AllPrimaryCarePatientsAge0to17                                  0
WellChildVisitsInPastYearAge0to17                               0
AsthmaDiagnosisAge0to17                                         0
YearOfStartDate                                                 0
UnderAge1PopEst2015_19ACS                                       0
Unique0to17WithED_Visit                                         0
NumberED_VisitsAge0to17                                         0
NumberLowAcuityED_VisitsAge0to17                                0
NumberAsthmaRelatedED_Visits                                    0
NumberED_VisitsByChildrenUnder1YearOld                          0
NumberOfInjuryRelatedVisits                                     0
NumberAcut

In [24]:
util_data

,Geo_FIPS,YearOfContactDate,TotalPopEst2015_19ACS_x,Age0to17PopEst2015_19ACS_x,AllPrimaryCarePatientsAge0to17,WellChildVisitsInPastYearAge0to17,AsthmaDiagnosisAge0to17,YearOfStartDate,UnderAge1PopEst2015_19ACS,Unique0to17WithED_Visit,NumberED_VisitsAge0to17,NumberLowAcuityED_VisitsAge0to17,NumberAsthmaRelatedED_Visits,NumberED_VisitsByChildrenUnder1YearOld,NumberOfInjuryRelatedVisits,NumberAcuteRespiratoryTractInfectionRelatedED_VisitsAge0to17
0,10300,2016,4147,155,50,33,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10300,2017,4147,155,41,24,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10300,2018,4147,155,44,34,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10300,2019,728,122,47,36,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,20100,2016,5490,201,71,44,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1580,981800,2019,207,47,4,3,2.0,2019.0,0.0,4.0,8.0,2.0,0.0,0.0,0.0,0.0
1581,982200,2016,4618,80,16,13,1.0,2016.0,0.0,6.0,10.0,2.0,0.0,1.0,0.0,0.0
1582,982200,2017,4618,80,18,16,1.0,2017.0,0.0,10.0,10.0,2.0,1.0,1.0,0.0,3.0
1583,982200,2018,4618,80,19,14,1.0,2018.0,0.0,7.0,9.0,2.0,0.0,2.0,0.0,3.0


### Race Data

In [25]:
#Census data has to be loaded in for each year in 2016-19
#Census indicates missing values using -
demo2016 = pd.read_csv('our-data/ACSDP5Y2016.DP05-Data.csv', header = 1, na_values='-')
#Adding in timeperiod indicator
demo2016['Year'] = [2016]*len(demo2016)
#Filtering out data that does not pertain to Census Tract identifiers, Median Age,
#or estimates for number of individuals from each racial group
demo2016 = demo2016[['Geography', 'Year',
          'Estimate!!SEX AND AGE!!Median age (years)',
          'Estimate!!RACE!!Total population',
          'Estimate!!RACE!!One race!!White',
          'Estimate!!RACE!!One race!!Black or African American',
          'Estimate!!RACE!!One race!!Asian',
          'Estimate!!RACE!!One race!!American Indian and Alaska Native',
          'Estimate!!RACE!!One race!!Native Hawaiian and Other Pacific Islander',
          'Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)',
          'Estimate!!RACE!!One race!!Some other race',
          'Estimate!!RACE!!Two or more races']]
#Renaming columns to be more easily legible
demo2016 = demo2016.rename(columns = {'Estimate!!SEX AND AGE!!Median age (years)':'Median Age',
                           'Estimate!!RACE!!Total population':'Total Population',
                           'Estimate!!RACE!!One race!!White':'White Population',
                           'Estimate!!RACE!!One race!!Black or African American':'Black or African American Population',
                           'Estimate!!RACE!!One race!!Asian':'Asian Population',
                           'Estimate!!RACE!!One race!!American Indian and Alaska Native': 'American Indian and Alaska Native Population',
                           'Estimate!!RACE!!One race!!Native Hawaiian and Other Pacific Islander':'Native Hawaiian and Other Pacific Islander Population',
                           'Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)':'Hispanic or Latino Population',
                           'Estimate!!RACE!!One race!!Some other race':'Some Other Race Population',
                           'Estimate!!RACE!!Two or more races':'Two or More Races Population'})

#Process is repeated for 2017, 2018, and 2019 but column names in original dataset change in these years
demo2017 = pd.read_csv('our-data/ACSDP5Y2017.DP05-Data.csv', header = 1, na_values='-')
demo2017['Year'] = [2017]*len(demo2017)
#Filtering reflects different name for columns. The real world value that is being estimated stays constant despite name change.
demo2017 = demo2017[['Geography', 'Year',
          'Estimate!!SEX AND AGE!!Total population!!Median age (years)',
          'Estimate!!RACE!!Total population',
          'Estimate!!RACE!!Total population!!One race!!White',
          'Estimate!!RACE!!Total population!!One race!!Black or African American',
          'Estimate!!RACE!!Total population!!One race!!Asian',
          'Estimate!!RACE!!Total population!!One race!!American Indian and Alaska Native',
          'Estimate!!RACE!!Total population!!One race!!Native Hawaiian and Other Pacific Islander',
          'Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)',
          'Estimate!!RACE!!Total population!!One race!!Some other race',
          'Estimate!!RACE!!Total population!!Two or more races']]
demo2017 = demo2017.rename(columns = {'Estimate!!SEX AND AGE!!Total population!!Median age (years)':'Median Age',
                           'Estimate!!RACE!!Total population':'Total Population',
                           'Estimate!!RACE!!Total population!!One race!!White':'White Population',
                           'Estimate!!RACE!!Total population!!One race!!Black or African American':'Black or African American Population',
                           'Estimate!!RACE!!Total population!!One race!!Asian':'Asian Population',
                           'Estimate!!RACE!!Total population!!One race!!American Indian and Alaska Native': 'American Indian and Alaska Native Population',
                           'Estimate!!RACE!!Total population!!One race!!Native Hawaiian and Other Pacific Islander':'Native Hawaiian and Other Pacific Islander Population',
                           'Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)':'Hispanic or Latino Population',
                           'Estimate!!RACE!!Total population!!One race!!Some other race':'Some Other Race Population',
                           'Estimate!!RACE!!Total population!!Two or more races':'Two or More Races Population'})

demo2018 = pd.read_csv('our-data/ACSDP5Y2018.DP05-Data.csv', header = 1, na_values='-')
demo2018['Year'] = [2018]*len(demo2018)
demo2018 = demo2018[['Geography', 'Year',
          'Estimate!!SEX AND AGE!!Total population!!Median age (years)',
          'Estimate!!RACE!!Total population',
          'Estimate!!RACE!!Total population!!One race!!White',
          'Estimate!!RACE!!Total population!!One race!!Black or African American',
          'Estimate!!RACE!!Total population!!One race!!Asian',
          'Estimate!!RACE!!Total population!!One race!!American Indian and Alaska Native',
          'Estimate!!RACE!!Total population!!One race!!Native Hawaiian and Other Pacific Islander',
          'Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)',
          'Estimate!!RACE!!Total population!!One race!!Some other race',
          'Estimate!!RACE!!Total population!!Two or more races']]
demo2018 = demo2018.rename(columns = {'Estimate!!SEX AND AGE!!Total population!!Median age (years)':'Median Age',
                           'Estimate!!RACE!!Total population':'Total Population',
                           'Estimate!!RACE!!Total population!!One race!!White':'White Population',
                           'Estimate!!RACE!!Total population!!One race!!Black or African American':'Black or African American Population',
                           'Estimate!!RACE!!Total population!!One race!!Asian':'Asian Population',
                           'Estimate!!RACE!!Total population!!One race!!American Indian and Alaska Native': 'American Indian and Alaska Native Population',
                           'Estimate!!RACE!!Total population!!One race!!Native Hawaiian and Other Pacific Islander':'Native Hawaiian and Other Pacific Islander Population',
                           'Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)':'Hispanic or Latino Population',
                           'Estimate!!RACE!!Total population!!One race!!Some other race':'Some Other Race Population',
                           'Estimate!!RACE!!Total population!!Two or more races':'Two or More Races Population'})

demo2019 = pd.read_csv('our-data/ACSDP5Y2019.DP05-Data.csv', header = 1, na_values='-')
demo2019['Year'] = [2019]*len(demo2019)
demo2019 = demo2019[['Geography', 'Year',
          'Estimate!!SEX AND AGE!!Total population!!Median age (years)',
          'Estimate!!RACE!!Total population',
          'Estimate!!RACE!!Total population!!One race!!White',
          'Estimate!!RACE!!Total population!!One race!!Black or African American',
          'Estimate!!RACE!!Total population!!One race!!Asian',
          'Estimate!!RACE!!Total population!!One race!!American Indian and Alaska Native',
          'Estimate!!RACE!!Total population!!One race!!Native Hawaiian and Other Pacific Islander',
          'Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)',
          'Estimate!!RACE!!Total population!!One race!!Some other race',
          'Estimate!!RACE!!Total population!!Two or more races']]
demo2019 = demo2019.rename(columns = {'Estimate!!SEX AND AGE!!Total population!!Median age (years)':'Median Age',
                           'Estimate!!RACE!!Total population':'Total Population',
                           'Estimate!!RACE!!Total population!!One race!!White':'White Population',
                           'Estimate!!RACE!!Total population!!One race!!Black or African American':'Black or African American Population',
                           'Estimate!!RACE!!Total population!!One race!!Asian':'Asian Population',
                           'Estimate!!RACE!!Total population!!One race!!American Indian and Alaska Native': 'American Indian and Alaska Native Population',
                           'Estimate!!RACE!!Total population!!One race!!Native Hawaiian and Other Pacific Islander':'Native Hawaiian and Other Pacific Islander Population',
                           'Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)':'Hispanic or Latino Population',
                           'Estimate!!RACE!!Total population!!One race!!Some other race':'Some Other Race Population',
                           'Estimate!!RACE!!Total population!!Two or more races':'Two or More Races Population'})

#Joining data for all four years to create one dataframe
demo_data = pd.concat([demo2016, demo2017, demo2018, demo2019], axis = 0)
demo_data

,Geography,Year,Median Age,Total Population,White Population,Black or African American Population,Asian Population,American Indian and Alaska Native Population,Native Hawaiian and Other Pacific Islander Population,Hispanic or Latino Population,Some Other Race Population,Two or More Races Population
0,1400000US42003010300,2016,21.6,6491,4433,1573,220,24,0,227,7,234
1,1400000US42003020100,2016,28.9,4212,3366,347,356,29,0,111,0,114
2,1400000US42003020300,2016,34.3,821,691,44,68,0,0,20,0,18
3,1400000US42003030500,2016,40.3,2292,198,1777,25,20,0,216,84,188
4,1400000US42003040200,2016,24.0,1609,571,807,153,0,19,18,0,59
...,...,...,...,...,...,...,...,...,...,...,...,...
397,1400000US42003981000,2019,NaN,0,0,0,0,0,0,0,0,0
398,1400000US42003981100,2019,NaN,0,0,0,0,0,0,0,0,0
399,1400000US42003981200,2019,NaN,0,0,0,0,0,0,0,0,0
400,1400000US42003981800,2019,19.4,207,113,55,17,0,0,17,10,12


In [26]:
#There are values for all 402 census tracts 35 rows are missing values for Median Age
demo_data.isna().sum()

Geography                                                 0
Year                                                      0
Median Age                                               35
Total Population                                          0
White Population                                          0
Black or African American Population                      0
Asian Population                                          0
American Indian and Alaska Native Population              0
Native Hawaiian and Other Pacific Islander Population     0
Hispanic or Latino Population                             0
Some Other Race Population                                0
Two or More Races Population                              0
dtype: int64

In [27]:
#Missing values are in census tract with either no population or very low population
demo_data[demo_data['Median Age'].isna()]

,Geography,Year,Median Age,Total Population,White Population,Black or African American Population,Asian Population,American Indian and Alaska Native Population,Native Hawaiian and Other Pacific Islander Population,Hispanic or Latino Population,Some Other Race Population,Two or More Races Population
390,1400000US42003980300,2016,NaN,0,0,0,0,0,0,0,0,0
391,1400000US42003980400,2016,NaN,0,0,0,0,0,0,0,0,0
393,1400000US42003980600,2016,NaN,4,4,0,0,0,0,0,0,0
395,1400000US42003980800,2016,NaN,0,0,0,0,0,0,0,0,0
397,1400000US42003981000,2016,NaN,0,0,0,0,0,0,0,0,0
398,1400000US42003981100,2016,NaN,5,5,0,0,0,0,0,0,0
399,1400000US42003981200,2016,NaN,0,0,0,0,0,0,0,0,0
388,1400000US42003980000,2017,NaN,0,0,0,0,0,0,0,0,0
390,1400000US42003980300,2017,NaN,0,0,0,0,0,0,0,0,0
391,1400000US42003980400,2017,NaN,0,0,0,0,0,0,0,0,0


### Household Income Data

In [28]:
#Census data has to be loaded in for each year in 2016-19
#Census indicates missing values using -
income2016 = pd.read_csv('our-data/ACSST5Y2016.S1903-Data.csv', header = 1, na_values='-')
#Adding time period indicator for each row
income2016['Year'] = [2016]*len(income2016)
#Filtering out columns that do not contain row identifier, number of households, or median income
income2016 = income2016[['Geography', 'Year', 'Total!!Estimate!!Households',
                         'Median income (dollars)!!Estimate!!Households']]
#Renaming columns to be more legible
income2016 = income2016.rename(columns={'Total!!Estimate!!Households':'Households',
                                        'Median income (dollars)!!Estimate!!Households':'Median Income'})

#Process is repeated for 2017, 2018, and 2019 but column names in original dataset change in these years
income2017 = pd.read_csv('our-data/ACSST5Y2017.S1903-Data.csv', header = 1, na_values='-')
income2017['Year'] = [2017]*len(income2017)
#Although names of columns change, same values are still being estimated
income2017 = income2017[['Geography', 'Year',
  'Estimate!!Number!!HOUSEHOLD INCOME BY RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER!!Households',
  'Estimate!!Median income (dollars)!!HOUSEHOLD INCOME BY RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER!!Households']]
income2017 = income2017.rename(columns={'Estimate!!Number!!HOUSEHOLD INCOME BY RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER!!Households':'Households',
                           'Estimate!!Median income (dollars)!!HOUSEHOLD INCOME BY RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER!!Households':'Median Income'})

income2018 = pd.read_csv('our-data/ACSST5Y2018.S1903-Data.csv', header = 1, na_values='-')
income2018['Year'] = [2018]*len(income2018)
income2018 = income2018[['Geography', 'Year',
  'Estimate!!Number!!HOUSEHOLD INCOME BY RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER!!Households',
  'Estimate!!Median income (dollars)!!HOUSEHOLD INCOME BY RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER!!Households']]
income2018 = income2018.rename(columns={'Estimate!!Number!!HOUSEHOLD INCOME BY RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER!!Households':'Households',
                           'Estimate!!Median income (dollars)!!HOUSEHOLD INCOME BY RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER!!Households':'Median Income'})

income2019 = pd.read_csv('our-data/ACSST5Y2019.S1903-Data.csv', header = 1, na_values='-')
income2019['Year'] = [2019]*len(income2019)
income2019 = income2019[['Geography', 'Year',
  'Estimate!!Number!!HOUSEHOLD INCOME BY RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER!!Households',
  'Estimate!!Median income (dollars)!!HOUSEHOLD INCOME BY RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER!!Households']]
income2019 = income2019.rename(columns={'Estimate!!Number!!HOUSEHOLD INCOME BY RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER!!Households':'Households',
                           'Estimate!!Median income (dollars)!!HOUSEHOLD INCOME BY RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER!!Households':'Median Income'})

#Joining data for all four years to create one dataframe
income_data=pd.concat([income2016,income2017,income2018,income2019])
income_data

,Geography,Year,Households,Median Income
0,1400000US42003010300,2016,281,22292.0
1,1400000US42003020100,2016,2042,72642.0
2,1400000US42003020300,2016,553,93125.0
3,1400000US42003030500,2016,1117,18730.0
4,1400000US42003040200,2016,611,16250.0
...,...,...,...,...
397,1400000US42003981000,2019,0,NaN
398,1400000US42003981100,2019,0,NaN
399,1400000US42003981200,2019,0,NaN
400,1400000US42003981800,2019,0,NaN


In [29]:
#Missing values for 50 census tracts
income_data.isna().sum()

Geography         0
Year              0
Households        0
Median Income    50
dtype: int64

In [30]:
#All but one of these census tracts has either no households or a very small number or households
income_data[income_data['Median Income'].isna()]

,Geography,Year,Households,Median Income
389,1400000US42003980100,2016,17,NaN
390,1400000US42003980300,2016,0,NaN
391,1400000US42003980400,2016,0,NaN
392,1400000US42003980500,2016,8,NaN
393,1400000US42003980600,2016,4,NaN
394,1400000US42003980700,2016,10,NaN
395,1400000US42003980800,2016,0,NaN
396,1400000US42003980900,2016,0,NaN
397,1400000US42003981000,2016,0,NaN
398,1400000US42003981100,2016,5,NaN


### Merging the two Census Datasets

In [31]:
census_data = demo_data.merge(income_data, how='inner', on=['Geography', 'Year'], validate='one_to_one')
#Creating indentifier that contains only census tract information
census_data['Geo_FIPS'] = census_data['Geography'].apply(lambda x: int(x[-6:]))
census_data

,Geography,Year,Median Age,Total Population,White Population,Black or African American Population,Asian Population,American Indian and Alaska Native Population,Native Hawaiian and Other Pacific Islander Population,Hispanic or Latino Population,Some Other Race Population,Two or More Races Population,Households,Median Income,Geo_FIPS
0,1400000US42003010300,2016,21.6,6491,4433,1573,220,24,0,227,7,234,281,22292.0,10300
1,1400000US42003020100,2016,28.9,4212,3366,347,356,29,0,111,0,114,2042,72642.0,20100
2,1400000US42003020300,2016,34.3,821,691,44,68,0,0,20,0,18,553,93125.0,20300
3,1400000US42003030500,2016,40.3,2292,198,1777,25,20,0,216,84,188,1117,18730.0,30500
4,1400000US42003040200,2016,24.0,1609,571,807,153,0,19,18,0,59,611,16250.0,40200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1603,1400000US42003981000,2019,NaN,0,0,0,0,0,0,0,0,0,0,NaN,981000
1604,1400000US42003981100,2019,NaN,0,0,0,0,0,0,0,0,0,0,NaN,981100
1605,1400000US42003981200,2019,NaN,0,0,0,0,0,0,0,0,0,0,NaN,981200
1606,1400000US42003981800,2019,19.4,207,113,55,17,0,0,17,10,12,0,NaN,981800


In [32]:
#Same values contain NaNs
census_data.isna().sum()

Geography                                                 0
Year                                                      0
Median Age                                               35
Total Population                                          0
White Population                                          0
Black or African American Population                      0
Asian Population                                          0
American Indian and Alaska Native Population              0
Native Hawaiian and Other Pacific Islander Population     0
Hispanic or Latino Population                             0
Some Other Race Population                                0
Two or More Races Population                              0
Households                                                0
Median Income                                            50
Geo_FIPS                                                  0
dtype: int64

### Emissions Data

Emissions data is weighted on inverse distance from facility producing them. See other notebook for process calculating values.

In [33]:
emissions_data = pd.read_csv('our-data/final_emissions_data.csv', index_col=0)
emissions_data

,TRACTCE,Year,"1,1,2-Trichloroethane","1,1,2,2-Tetrachloroethane","1,2-Ethanediol (Ethylene Glycol)","1,3-Butadiene","1,4-Dichlorobenzene","1,4-Dioxane (1,4-Diethyleneoxide)","2,2,4-Trimethylpentane",Acetaldehyde,Acrolein,Acrylic Acid,Acrylonitrile,Ammonia,Antimony,Arsenic,Benzene,Cadmium,Carbon Dioxide,Carbon Disulfide,Carbon Monoxide,Carbon Tetrachloride,Chlorine,Chlorobenzene,Chloroethene (vinyl chloride),Chloroform,Chromium,Cobalt,Cresols/Cresylic Acid (Isomers And Mixture),Cyanides,Dibutylphthalate,Ethyl Chloride (Chloroethane),Ethylbenzene,"Ethylene Dichloride (1,2-Dichloroethane)",Ethylene Oxide,"Ethylidene Dichloride (1,1-Dichloroethane)",Formaldehyde,Glycol Ethers,Hexane,Hexavalent Chromium,Hydrochloric Acid,Hydrogen Fluoride (Hydrofluoric Acid),Hydrogen Sulfide,Isophorone,Lead,m-Xylene,Maleic Anhydride,Manganese,Mercury,Methane,Methanol,Methyl Chloride (Chloromethane),"Methyl Chloroform (1,1,1-Trichloroethane)",Methyl Isobutyl Ketone (4-Methyl-2-Pentanone),Methyl Methacrylate,Methylene Chloride (Dichloromethane),Naphthalene,Nickel,Nitrogen Oxides,Nitrous Oxide (N2O),o-Xylene,"Particulate Matter < 10 Microns, Filterable","Particulate Matter < 2.5 Microns, Filterable","Particulate Matter, Condensable",Phenol,Phosphorus,Phthalic Anhydride,Polycyclic Organic Matter,Propionaldehyde,Propylene Oxide,Selenium,Styrene,Sulfur Oxides,Sulfuric Acid,Tetrachloroethylene (Perchloroethylene),Toluene,Trichloroethylene,"Vinylidene Chloride (1,1-Dichloroethylene)",Volatile Organic Compounds,Xylenes (Isomers And Mixture),Grand Total
0,50900,2016.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.016377,0.001580,0.0,0.0,0.0,0.847486,0.0,0.0,0.072221,0.0,51097.646442,0.000000,22.786588,0.0,0.000000,0.0,0.0,0.320509,0.000000,0.0,0.043571,0.0,0.0,0.0,0.004132,0.0,0.002420,0.0,0.082861,0.000000,0.958332,0.0,0.000000,0.0,0.0,0.0,0.000347,0.0,0.0,0.001245,0.000025,1.501119,0.037409,0.000000,0.0,0.000000,0.0,0.110522,0.032151,0.0,42.735675,0.142162,0.0,1.607965,0.803724,1.135155,0.046061,0.000000,0.0,0.0,0.0,0.001580,0.0,0.0,0.987008,0.0,0.0,0.155232,0.270461,0.0,9.035630,0.168572,51181.554563
1,70300,2016.0,0.0,0.0,0.011032,0.0,0.0,0.0,0.017433,0.000826,0.0,0.0,0.0,0.485450,0.0,0.0,0.060666,0.0,32090.112989,0.000000,15.331787,0.0,0.000000,0.0,0.0,0.210637,0.000000,0.0,0.037500,0.0,0.0,0.0,0.003058,0.0,0.002245,0.0,0.049352,0.011032,0.615750,0.0,0.000000,0.0,0.0,0.0,0.000237,0.0,0.0,0.001071,0.000014,1.003526,0.020180,0.000000,0.0,0.000000,0.0,0.072140,0.032204,0.0,23.620214,0.085491,0.0,1.286679,0.649022,0.975043,0.039642,0.000000,0.0,0.0,0.0,0.000826,0.0,0.0,0.756171,0.0,0.0,0.113205,0.532497,0.0,8.771340,0.375022,32145.284280
2,120700,2016.0,0.0,0.0,0.026404,0.0,0.0,0.0,0.015003,0.000000,0.0,0.0,0.0,0.055593,0.0,0.0,0.035833,0.0,8063.066062,0.000000,5.236010,0.0,0.000000,0.0,0.0,0.126364,0.000000,0.0,0.028028,0.0,0.0,0.0,0.000000,0.0,0.001514,0.0,0.002590,0.026404,0.188151,0.0,0.000000,0.0,0.0,0.0,0.000092,0.0,0.0,0.000801,0.000007,0.150202,0.000000,0.000000,0.0,0.000000,0.0,0.042121,0.031620,0.0,8.174284,0.015676,0.0,0.535142,0.249512,0.439943,0.029629,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.395723,0.0,0.0,0.071339,1.127911,0.0,8.155370,0.782580,8089.009905
3,140400,2016.0,0.0,0.0,0.021126,0.0,0.0,0.0,0.014855,0.000000,0.0,0.0,0.0,0.245791,0.0,0.0,0.047832,0.0,17727.250155,0.000000,9.869439,0.0,0.000000,0.0,0.0,0.140321,0.000000,0.0,0.029560,0.0,0.0,0.0,0.002362,0.0,0.001650,0.0,0.026465,0.021126,0.351599,0.0,0.000000,0.0,0.0,0.0,0.000164,0.0,0.0,0.000845,0.000009,0.639982,0.004465,0.000000,0.0,0.000000,0.0,0.046774,0.031476,0.0,12.803571,0.052175,0.0,0.973099,0.478228,0.684856,0.031249,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.535702,0.0,0.0,0.084212,0.803240,0.0,8.441650,0.578921,17764.212898
4,180700,2016.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.001626,0.0,0.0,0.0,0.516487,0.0,0.0,0.026667,0.0,33204.813779,0.000000,15.805098,0.0,0.000000,0.0,0.0,0.159611,0.000000,0.0,0.000000,0.0,0.0,0.0,0.003131,0.0,0.000000,0.0,0.071280,0.000000,0.606706,0.0,0.000000,0.0

### Final Merges

In [34]:
#Merging health care utlization data with census data on race and household income
census_data['Geo_FIPS'] = census_data['Geo_FIPS'].astype(int)
utilxcensus_data = util_data.merge(census_data, how='left', left_on=['Geo_FIPS','YearOfContactDate'], right_on=['Geo_FIPS','Year'])
utilxcensus_data['Geo_FIPS']=utilxcensus_data['Geo_FIPS'].apply(int)
utilxcensus_data

,Geo_FIPS,YearOfContactDate,TotalPopEst2015_19ACS_x,Age0to17PopEst2015_19ACS_x,AllPrimaryCarePatientsAge0to17,WellChildVisitsInPastYearAge0to17,AsthmaDiagnosisAge0to17,YearOfStartDate,UnderAge1PopEst2015_19ACS,Unique0to17WithED_Visit,NumberED_VisitsAge0to17,NumberLowAcuityED_VisitsAge0to17,NumberAsthmaRelatedED_Visits,NumberED_VisitsByChildrenUnder1YearOld,NumberOfInjuryRelatedVisits,NumberAcuteRespiratoryTractInfectionRelatedED_VisitsAge0to17,Geography,Year,Median Age,Total Population,White Population,Black or African American Population,Asian Population,American Indian and Alaska Native Population,Native Hawaiian and Other Pacific Islander Population,Hispanic or Latino Population,Some Other Race Population,Two or More Races Population,Households,Median Income
0,10300,2016,4147,155,50,33,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1400000US42003010300,2016,21.6,6491,4433,1573,220,24,0,227,7,234,281,22292.0
1,10300,2017,4147,155,41,24,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1400000US42003010300,2017,21.8,6546,4299,1738,246,27,0,222,6,230,291,22176.0
2,10300,2018,4147,155,44,34,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1400000US42003010300,2018,21.8,6668,4228,1871,239,22,0,240,14,294,320,21136.0
3,10300,2019,728,122,47,36,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1400000US42003010300,2019,21.8,6652,4088,1948,229,28,0,254,21,338,322,23603.0
4,20100,2016,5490,201,71,44,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1400000US42003020100,2016,28.9,4212,3366,347,356,29,0,111,0,114,2042,72642.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1580,981800,2019,207,47,4,3,2.0,2019.0,0.0,4.0,8.0,2.0,0.0,0.0,0.0,0.0,1400000US42003981800,2019,19.4,207,113,55,17,0,0,17,10,12,0,NaN
1581,982200,2016,4618,80,16,13,1.0,2016.0,0.0,6.0,10.0,2.0,0.0,1.0,0.0,0.0,1400000US42003982200,2016,19.3,4742,3963,276,341,26,0,61,10,126,22,16250.0
1582,982200,2017,4618,80,18,16,1.0,2017.0,0.0,10.0,10.0,2.0,1.0,1.0,0.0,3.0,1400000US42003982200,2017,19.3,4644,3759,260,433,50,0,109,18,124,19,14250.0
1583,982200,2018,4618,80,19,14,1.0,2018.0,0.0,7.0,9.0,2.0,0.0,2.0,0.0,3.0,1400000US42003982200,2018,19.3,4619,3831,249,381,0,0,64,17,141,11,169063.0


In [35]:
#merged census and utilization data with health condition prevalence data
utilxcensus_data = utilxcensus_data.dropna(subset=['YearOfContactDate'])
utilxcensus_data['YearOfContactDate'] = utilxcensus_data['YearOfContactDate'].astype(int)
prevalence_data['Year']= prevalence_data['Year'].astype(int)
semi_data = utilxcensus_data.merge(prevalence_data, how='right', left_on=['Geo_FIPS', 'Year'], right_on=['Geo_FIPS','Year'])
semi_data

,Geo_FIPS,YearOfContactDate,TotalPopEst2015_19ACS_x,Age0to17PopEst2015_19ACS_x,AllPrimaryCarePatientsAge0to17,WellChildVisitsInPastYearAge0to17,AsthmaDiagnosisAge0to17,YearOfStartDate,UnderAge1PopEst2015_19ACS,Unique0to17WithED_Visit,NumberED_VisitsAge0to17,NumberLowAcuityED_VisitsAge0to17,NumberAsthmaRelatedED_Visits,NumberED_VisitsByChildrenUnder1YearOld,NumberOfInjuryRelatedVisits,NumberAcuteRespiratoryTractInfectionRelatedED_VisitsAge0to17,Geography,Year,Median Age,Total Population,White Population,Black or African American Population,Asian Population,American Indian and Alaska Native Population,Native Hawaiian and Other Pacific Islander Population,Hispanic or Latino Population,Some Other Race Population,Two or More Races Population,Households,Median Income,ACCESS2_CrudePrev,BPHIGH_CrudePrev,BPMED_CrudePrev,CASTHMA_CrudePrev,CHD_CrudePrev,CHECKUP_CrudePrev,CHOLSCREEN_CrudePrev,COPD_CrudePrev,COREM_CrudePrev,CSMOKING_CrudePrev,DIABETES_CrudePrev,HIGHCHOL_CrudePrev,OBESITY_CrudePrev,Geolocation,lat,lon
0,10300,2019.0,728.0,122.0,47.0,36.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1400000US42003010300,2019,21.8,6652.0,4088.0,1948.0,229.0,28.0,0.0,254.0,21.0,338.0,322.0,23603.0,12.9,19.0,51.9,11.1,2.6,63.7,58.1,4.4,38.8,26.5,5.0,17.6,29.8,"(40.43572601140, -79.9851241394)",40.43572601140,-79.9851241394
1,20100,2019.0,5490.0,201.0,67.0,48.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1400000US42003020100,2019,29.7,5490.0,4320.0,472.0,527.0,23.0,0.0,272.0,0.0,148.0,2714.0,77759.0,6.7,22.1,71.8,9.3,3.9,68.4,73.4,3.8,50.2,14.4,6.4,24.9,23.4,"(40.44068353060, -80.0000351595)",40.44068353060,-80.0000351595
2,20300,2019.0,1212.0,41.0,38.0,39.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1400000US42003020300,2019,31.7,1212.0,1050.0,23.0,121.0,0.0,0.0,36.0,0.0,18.0,821.0,106313.0,4.8,19.7,67.6,8.2,2.8,67.4,80.4,2.8,54.3,12.3,4.9,24.2,23.4,"(40.45425423370, -79.9814998549)",40.45425423370,-79.9814998549
3,30500,2019.0,2109.0,381.0,280.0,219.0,49.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1400000US42003030500,2019,40.4,2109.0,294.0,1628.0,40.0,2.0,0.0,162.0,0.0,145.0,1145.0,19243.0,14.4,48.3,84.3,12.8,9.9,79.2,79.7,10.6,32.5,26.9,20.5,37.2,42.6,"(40.44273346300, -79.9837719110)",40.44273346300,-79.9837719110
4,40200,2019.0,1576.0,100.0,68.0,50.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1400000US42003040200,2019,23.5,1576.0,815.0,540.0,168.0,0.0,0.0,56.0,30.0,23.0,600.0,18500.0,11.8,23.3,67.1,11.7,4.0,67.5,61.4,5.5,31.8,22.3,8.0,22.3,29.3,"(40.44011022030, -79.9663476233)",40.44011022030,-79.9663476233
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1563,564000,2016.0,6590.0,1597.0,511.0,428.0,54.0,6048.0,90.6,155.0,196.0,57.0,23.0,18.0,2.0,12.0,1400000US42003564000,2016,41.1,5949.0,5735.0,33.0,138.0,37.0,0.0,25.0,6.0,0.0,2374.0,64505.0,8.3,28.1,74.5,9.9,5.6,77.1,85.8,6.4,43.9,19.7,8.4,32.3,31.9,POINT (-80.2402315 40.41285776),OINT (-80.2402,15 40.41285776
1564,564100,2016.0,883.0,181.0,32.0,30.0,5.0,4032.0,7.8,15.0,16.0,5.0,1.0,1.0,0.0,1.0,1400000US42003564100,2016,49.0,914.0,885.0,5.0,12.0,0.0,0.0,7.0,0.0,12.0,368.0,131875.0,4.3,28.8,79.0,8.7,5.4,81.1,91.1,4.7,50.8,10.8,8.0,34.8,28.1,POINT (-80.08698899 40.42759244),OINT (-80.0869,99 40.42759244
1565,564200,2016.0,2091.0,272.0,123.0,101.0,19.0,6048.0,6.6,48.0,70.0,12.0,25.0,5.0,0.0,9.0,1400000US42003564200,2016,53.2,2212.0,2063.0,27.0,44.0,0.0,0.0,25.0,0.0,78.0,1076.0,57857.0,7.6,33.4,79.7,9.6,7.2,80.0,88.0,7.4,43.7,18.1,10.3,36.6,31.5,POINT (-79.79426581 40.33230116),OINT (-79.7942,81 40.33230116
1566,564400,2016.0,5874.0,910.0,408.0,341.0,57.0,14112.0,48.2,188.0,315.0,65.0,67.0,48.0,4.0,38.0,1400000US42003564400,2016,46.9,5508.0,4999.0,411.0,0.0,2.0,0.0,39.0,18.0,78.0,2657.0,44298.0,8.7,32.8,78.5,10.1,7.2,79.3,86.8,7.7,42.5,20.1,10.4,35.5,32.7,POINT (-79.79127061 40.37623919),OINT (-79.7912,61 40.37623919


In [36]:
#joining emissions dataset
data = semi_data.merge(emissions_data, how='inner', left_on=['Geo_FIPS', 'YearOfContactDate'], right_on=['TRACTCE','Year'])
data

,Geo_FIPS,YearOfContactDate,TotalPopEst2015_19ACS_x,Age0to17PopEst2015_19ACS_x,AllPrimaryCarePatientsAge0to17,WellChildVisitsInPastYearAge0to17,AsthmaDiagnosisAge0to17,YearOfStartDate,UnderAge1PopEst2015_19ACS,Unique0to17WithED_Visit,NumberED_VisitsAge0to17,NumberLowAcuityED_VisitsAge0to17,NumberAsthmaRelatedED_Visits,NumberED_VisitsByChildrenUnder1YearOld,NumberOfInjuryRelatedVisits,NumberAcuteRespiratoryTractInfectionRelatedED_VisitsAge0to17,Geography,Year_x,Median Age,Total Population,White Population,Black or African American Population,Asian Population,American Indian and Alaska Native Population,Native Hawaiian and Other Pacific Islander Population,Hispanic or Latino Population,Some Other Race Population,Two or More Races Population,Households,Median Income,ACCESS2_CrudePrev,BPHIGH_CrudePrev,BPMED_CrudePrev,CASTHMA_CrudePrev,CHD_CrudePrev,CHECKUP_CrudePrev,CHOLSCREEN_CrudePrev,COPD_CrudePrev,COREM_CrudePrev,CSMOKING_CrudePrev,DIABETES_CrudePrev,HIGHCHOL_CrudePrev,OBESITY_CrudePrev,Geolocation,lat,lon,TRACTCE,Year_y,"1,1,2-Trichloroethane","1,1,2,2-Tetrachloroethane","1,2-Ethanediol (Ethylene Glycol)","1,3-Butadiene","1,4-Dichlorobenzene","1,4-Dioxane (1,4-Diethyleneoxide)","2,2,4-Trimethylpentane",Acetaldehyde,Acrolein,Acrylic Acid,Acrylonitrile,Ammonia,Antimony,Arsenic,Benzene,Cadmium,Carbon Dioxide,Carbon Disulfide,Carbon Monoxide,Carbon Tetrachloride,Chlorine,Chlorobenzene,Chloroethene (vinyl chloride),Chloroform,Chromium,Cobalt,Cresols/Cresylic Acid (Isomers And Mixture),Cyanides,Dibutylphthalate,Ethyl Chloride (Chloroethane),Ethylbenzene,"Ethylene Dichloride (1,2-Dichloroethane)",Ethylene Oxide,"Ethylidene Dichloride (1,1-Dichloroethane)",Formaldehyde,Glycol Ethers,Hexane,Hexavalent Chromium,Hydrochloric Acid,Hydrogen Fluoride (Hydrofluoric Acid),Hydrogen Sulfide,Isophorone,Lead,m-Xylene,Maleic Anhydride,Manganese,Mercury,Methane,Methanol,Methyl Chloride (Chloromethane),"Methyl Chloroform (1,1,1-Trichloroethane)",Methyl Isobutyl Ketone (4-Methyl-2-Pentanone),Methyl Methacrylate,Methylene Chloride (Dichloromethane),Naphthalene,Nickel,Nitrogen Oxides,Nitrous Oxide (N2O),o-Xylene,"Particulate Matter < 10 Microns, Filterable","Particulate Matter < 2.5 Microns, Filterable","Particulate Matter, Condensable",Phenol,Phosphorus,Phthalic Anhydride,Polycyclic Organic Matter,Propionaldehyde,Propylene Oxide,Selenium,Styrene,Sulfur Oxides,Sulfuric Acid,Tetrachloroethylene (Perchloroethylene),Toluene,Trichloroethylene,"Vinylidene Chloride (1,1-Dichloroethylene)",Volatile Organic Compounds,Xylenes (Isomers And Mixture),Grand Total
0,10300,2019.0,728.0,122.0,47.0,36.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1400000US42003010300,2019,21.8,6652.0,4088.0,1948.0,229.0,28.0,0.0,254.0,21.0,338.0,322.0,23603.0,12.9,19.0,51.9,11.1,2.6,63.7,58.1,4.4,38.8,26.5,5.0,17.6,29.8,"(40.43572601140, -79.9851241394)",40.43572601140,-79.9851241394,10300,2019.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,1.107472,0.000000,0.0,0.071123,0.0,57746.648057,0.000000,26.749009,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.029811,0.0,0.0,0.0,0.004973,0.000000,0.0,0.00000,0.287943,0.000000,1.797760,0.0,0.029872,0.0,0.000000,0.0,0.013178,0.000000,0.0,0.000852,0.000805,1.773751,0.534507,0.002517,0.000000,0.030203,0.00000,0.767501,0.019416,0.000852,51.606413,0.250957,0.000000,5.356027,3.162408,1.848928,0.032367,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,1.019873,0.0,0.000000,0.211597,0.000000,0.0,9.693386,0.117317,57853.168877
1,20100,2019.0,5490.0,201.0,67.0,48.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1400000US42003020100,2019,29.7,5490.0,4320.0,472.0,527.0,23.0,0.0,272.0,0.0,148.0,2714.0,77759.0,6.7,22.1,71.8,9.3,3.9,68.4,73.4,3.8,50.2,14.4,6.4,24.9,23.4,"(40.44068353060, -80.0000351595)",40.44068353060,-80.0000351595,20100,2019.0,0.007564,0.000000,0.000000,0.0,0.052951,0.000000,0.000000,0.002521,0.000000,0.0,0.00000,1.119707,0.000000,0.0,0.065642,0.0,75835.565289,0.036561,47.329617,0.0,0.000000,0.001261,

In [39]:
#Filtering out columns that are not important and then downloading csv
cols = ['Geo_FIPS','YearOfContactDate',
 'TotalPopEst2015_19ACS_x',
 'Age0to17PopEst2015_19ACS_x',
 'AllPrimaryCarePatientsAge0to17',
 'WellChildVisitsInPastYearAge0to17',
 'AsthmaDiagnosisAge0to17',
 'UnderAge1PopEst2015_19ACS',
 'Unique0to17WithED_Visit',
 'NumberED_VisitsAge0to17',
 'NumberLowAcuityED_VisitsAge0to17',
 'NumberAsthmaRelatedED_Visits',
 'NumberED_VisitsByChildrenUnder1YearOld',
 'NumberOfInjuryRelatedVisits',
 'NumberAcuteRespiratoryTractInfectionRelatedED_VisitsAge0to17',
 'Median Age',
 'Total Population',
 'White Population',
 'Black or African American Population',
 'Asian Population',
 'American Indian and Alaska Native Population',
 'Native Hawaiian and Other Pacific Islander Population',
 'Hispanic or Latino Population',
 'Some Other Race Population',
 'Two or More Races Population',
 'Households',
 'Median Income',
 'ACCESS2_CrudePrev',
 'BPHIGH_CrudePrev',
 'BPMED_CrudePrev',
 'CASTHMA_CrudePrev',
 'CHD_CrudePrev',
 'CHECKUP_CrudePrev',
 'CHOLSCREEN_CrudePrev',
 'COPD_CrudePrev',
 'COREM_CrudePrev',
 'CSMOKING_CrudePrev',
 'DIABETES_CrudePrev',
 'HIGHCHOL_CrudePrev',
 'OBESITY_CrudePrev',
 '1,1,2-Trichloroethane',
 '1,1,2,2-Tetrachloroethane',
 '1,2-Ethanediol (Ethylene Glycol)',
 '1,3-Butadiene',
 '1,4-Dichlorobenzene',
 '1,4-Dioxane (1,4-Diethyleneoxide)',
 '2,2,4-Trimethylpentane',
 'Acetaldehyde',
 'Acrolein',
 'Acrylic Acid',
 'Acrylonitrile',
 'Ammonia',
 'Antimony',
 'Arsenic',
 'Benzene',
 'Cadmium',
 'Carbon Dioxide',
 'Carbon Disulfide',
 'Carbon Monoxide',
 'Carbon Tetrachloride',
 'Chlorine',
 'Chlorobenzene',
 'Chloroethene (vinyl chloride)',
 'Chloroform',
 'Chromium',
 'Cobalt',
 'Cresols/Cresylic Acid (Isomers And Mixture)',
 'Cyanides',
 'Dibutylphthalate',
 'Ethyl Chloride (Chloroethane)',
 'Ethylbenzene',
 'Ethylene Dichloride (1,2-Dichloroethane)',
 'Ethylene Oxide',
 'Ethylidene Dichloride (1,1-Dichloroethane)',
 'Formaldehyde',
 'Glycol Ethers',
 'Hexane',
 'Hexavalent Chromium',
 'Hydrochloric Acid',
 'Hydrogen Fluoride (Hydrofluoric Acid)',
 'Hydrogen Sulfide',
 'Isophorone',
 'Lead',
 'm-Xylene',
 'Maleic Anhydride',
 'Manganese',
 'Mercury',
 'Methane',
 'Methanol',
 'Methyl Chloride (Chloromethane)',
 'Methyl Chloroform (1,1,1-Trichloroethane)',
 'Methyl Isobutyl Ketone (4-Methyl-2-Pentanone)',
 'Methyl Methacrylate',
 'Methylene Chloride (Dichloromethane)',
 'Naphthalene',
 'Nickel',
 'Nitrogen Oxides',
 'Nitrous Oxide (N2O)',
 'o-Xylene',
 'Particulate Matter < 10 Microns, Filterable',
 'Particulate Matter < 2.5 Microns, Filterable',
 'Particulate Matter, Condensable',
 'Phenol',
 'Phosphorus',
 'Phthalic Anhydride',
 'Polycyclic Organic Matter',
 'Propionaldehyde',
 'Propylene Oxide',
 'Selenium',
 'Styrene',
 'Sulfur Oxides',
 'Sulfuric Acid',
 'Tetrachloroethylene (Perchloroethylene)',
 'Toluene',
 'Trichloroethylene',
 'Vinylidene Chloride (1,1-Dichloroethylene)',
 'Volatile Organic Compounds',
 'Xylenes (Isomers And Mixture)',
 'Grand Total']
data = data[cols]
data.to_csv('data.csv')

In [41]:
#Loading in Pre1950s Housing values for additional features
pre1950housing = pd.read_csv('our-data/pre1950housing.xls-pre1950housing.csv')
h2016 = pre1950housing.copy()
h2016['Year'] = [2016]*len(h2016)
h2017 = pre1950housing.copy()
h2017['Year'] = [2017]*len(h2017)
h2018 = pre1950housing.copy()
h2018['Year'] = [2018]*len(h2018)
h2019 = pre1950housing.copy()
h2019['Year'] = [2019]*len(h2019)
pre1950 = pd.concat([h2016, h2017, h2018, h2019])
pre1950
test = data.merge(pre1950, how = 'left', left_on=['Geo_FIPS','YearOfContactDate'], right_on=['TRACT','Year'])
test.to_csv('our-data/data_W_1950s.csv')